# Solar Panel Detection using Mask RCNN

In [ ]:
# !pip install pyyaml==5.1
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
# assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

In [ ]:
# Assumption: matplotlib, numpy, opencv are installed

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import os, json, cv2, random
from PIL import Image
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
img = Image.open('10sfg735670.tif')
img = np.array(img)
plt.imshow(img)
plt.show()

#### An example ground truth bounding box

In [ ]:
plt.imshow(img[570:608, 2000:2030, :])
plt.show()

## Train on a custom dataset

In [ ]:
# download, decompress the data
labels = json.load(open('SolarArrayPolygons.json', 'r'))

In [ ]:
relevant_items = [item for item in labels['polygons'] if item['image_name'] == '10sfg735670']

### Register train/val/test datasets (converting arbitrary dataset formats to COCO format)

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode

def get_solar_dicts():
    with open("SolarArrayPolygons.json") as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    filenames = ['10sfg735670']
    
    for filename in filenames:
        record = {}
        
        img = Image.open(filename + '.tif')
        img = np.array(img)
        height, width = img.shape[:2]
        
        record["file_name"] = filename + '.tif'
        record["image_id"] = 0
        record["height"] = height
        record["width"] = width
        
        relevant_items = [item for item in imgs_anns['polygons'] if item['image_name'] == filename]
      
        objs = []
        for item in relevant_items:
            vertices = item["polygon_vertices_pixels"]
            px = [vertex[0] for vertex in vertices]
            py = [vertex[1] for vertex in vertices]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,  # only one single object class (solar)
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train"]:
    DatasetCatalog.register("solar_" + d, lambda d=d: get_solar_dicts())
    MetadataCatalog.get("solar_" + d).set(thing_classes=["solar"])
solar_metadata = MetadataCatalog.get("solar_train")

In [ ]:
solar_metadata

In [ ]:
dataset_dicts = get_solar_dicts()
for d in random.sample(dataset_dicts, 1):
    visualizer = Visualizer(img, metadata=solar_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    print(out.get_image().shape)
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    ax.imshow(out.get_image())
    imgPIL = Image.fromarray(out.get_image())
    imgPIL.size
    imgPIL.save('sample_gt_output.tif')
    plt.show()

## Training

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

class CocoTrainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

### Instantiate a Mask-RCNN config object

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("solar_train",)
cfg.DATASETS.VAL = ("solar_train",)
cfg.DATASETS.TEST = ("solar_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 301    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.TEST.EVAL_PERIOD = 100
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (solar). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

#### Actual training step

In [ ]:
%%time
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# # Look at training curves in tensorboard:
# %load_ext tensorboard
# %tensorboard --logdir output

## Inference/Evaluation

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.DATASETS.TEST = ("solar_train",)
cfg.DATASETS.VAL = ("solar_train",)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_solar_dicts()
for d in random.sample(dataset_dicts, 1):
    img = Image.open(d["file_name"])
    img = np.array(img)
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(img,
                   metadata=solar_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image())
    plt.show()
    print(outputs["instances"].to("cpu"))

In [ ]:
imgPIL = Image.fromarray(out.get_image())
imgPIL.size
imgPIL.save('sample_pred_maskrcnn_output.tif')

### Instantiate a Faster-RCNN config object

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("solar_train",)
cfg.DATASETS.VAL = ("solar_train",)
cfg.DATASETS.TEST = ("solar_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 601    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.TEST.EVAL_PERIOD = 100
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (solar). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

#### Actual training

In [ ]:
%%time
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Inference/Evaluation

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.DATASETS.TEST = ("solar_train",)
cfg.DATASETS.VAL = ("solar_train",)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_solar_dicts()
for d in random.sample(dataset_dicts, 1):
    img = Image.open(d["file_name"])
    img = np.array(img)
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs["instances"])
    v = Visualizer(img,
                   metadata=solar_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image())
    plt.show()
    print(outputs["instances"].to("cpu"))

In [ ]:
imgPIL = Image.fromarray(out.get_image())
imgPIL.size
imgPIL.save('sample_pred_fasterrcnn_output.tif')